In [1]:
pwd

'/mnt/4TB/TCGA_Colorectal/scripts/3-processing'

In [2]:
from guppy import hpy

In [3]:
print("start")
import sys, os
from os.path import join as pj
import pandas as pd

# ====== PROJECT SCAFFOLD ======
# Arguments passed into script by Docker command
project = sys.argv[1]
file_id = sys.argv[2]

# Mounted volume
wd = '/mnt/4TB/TCGA_Colorectal/scripts/3-processing'
beep = pj(wd, '43cbb4c9-4dc8-4e7a-ac69-779f5a76dced.pq')

df = pd.read_parquet(beep)

# Variant must be mapped to a gene
df = df[df['VEP_SYMBOL']!='']

# Fill empty AF with 0
df['VEP_gnomADg_AF'].replace('',0,inplace=True)

# Fill empty with 1
df['VEP_am_pathogenicity'].replace('',1,inplace=True)

df = df.astype({
    'VEP_gnomADg_AF':'float64',
    'VEP_am_pathogenicity':'float64',
    'FORMAT_GQ':'int64',
    'FORMAT_DP':'int64',
})

# Need to adjust pipeline to handle allosomal dosage and homologous PAR
df = df[(df['CHROM']!='chrX') & (df['CHROM']!='chrY')]


df = df[(df['VEP_IMPACT']=='HIGH') | (df['VEP_IMPACT']=='MODERATE')]

# 99% of of low quality (GQ<10) variants are very low depth (DP<5)
# ACMG recommends depth 10, but there are a decent amount high GQ variants between DP 5-10
# It would be hard to justify a high-priority feature with very low depth
df = df[df['FORMAT_DP']>=5]


# All of the non-missense will be weighted by allele freq as planned
# The MODERATE-impact indels are as rare as HIGH-impact frameshift mutations, and "small" indels can be 1,000 bp long
# There is a lot of evidence suggesting 0.1% is the cutoff, and much of the high-impact variation is common
# ^ https://docs.google.com/spreadsheets/d/1E7ZQrYo5kXu57rRl9zaSyu3W11cqb_qLjRuc309JMXg/edit#gid=1966751467

# AlphaMissense claims to accounts for not only structure, but also human/primate evolutionary conservation (rare/common AF)
# However, there are loads of extremely high AF (0.20-1.0) variants marked as pathogenic
afs     = df['VEP_gnomADg_AF'].tolist()
qc_afs  = []
for af in afs:
    qc_af = (1-af)**40
    qc_af = round(qc_af, 5)
    qc_afs.append(qc_af)
df['QC_AF'] = qc_afs



# GQ is a phred score = https://gatk.broadinstitute.org/hc/en-us/articles/360035531872
# We want it's likelihood 0.0-1.0
gqs    = df['FORMAT_GQ'].tolist()
qc_gqs = []
for gq in gqs:
    qc_gq = 1-10**-(gq/10)
    qc_gqs.append(qc_gq)
df['QC_GQ'] = qc_gqs


# PL docs = https://gatk.broadinstitute.org/hc/en-us/articles/360035890451
gts    = df['FORMAT_GT'].tolist()
pls    = df['FORMAT_PL'].tolist()
gqs    = df['QC_GQ'].tolist()
qc_gts = []
for e, gt in enumerate(gts):
    # Either way, it's a hom
    if (gt=='1/2'): qc_gt=2.0
    
    # There is a chance that this hom is actually a het
    elif (gt=='1/1'): qc_gt=2.0-(1-gqs[e])
    
    # Does it lean toward hom or no variation? Check its PL values
    # Note that `1/2` has 6 PL values, so making PL a column would be tricky
    elif (gt=='0/1'):
        pl = pls[e].split(',')
        pl0, pl2 = pl[0], pl[2]
        
        # e.g. [2,0,2] doesn't lean either way, so assume it's a het
        if (pl0==pl2): qc_gt=1.0
            
        # Otherwise, does uncertainty lean toward either hom or no variation?
        else:
            pl0, pl2 = int(pl0), int(pl2)
            # e.g. [8276,0,6] chance the variant is a hom
            if (pl0>pl2): qc_gt=2-gqs[e]
            #e.g. [5,0,165] chance the variant does not exist
            else: qc_gt=gqs[e]
    else:
        msg = f"Error - unknown genotype:{gt}"
    qc_gts.append(qc_gt)
df['QC_GT'] = qc_gts

# Compile it into a weight column
df['QC_Weight'] = df['QC_AF']*df['QC_GT']

df.reset_index(inplace=True,drop=True)

h = hpy()
print(h.heap())

# df.to_parquet(efs_qc)
print("end")

start
Partition of a set of 604766 objects. Total size = 123713725 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0      1   0 43824841  35  43824841  35 pandas.core.frame.DataFrame
     1 162823  27 21354406  17  65179247  53 str
     2 148504  25 12485272  10  77664519  63 tuple
     3     83   0  6059214   5  83723733  68 numpy.ndarray
     4  14919   2  5262848   4  88986581  72 dict (no owner)
     5  66008  11  5015670   4  94002251  76 bytes
     6  33471   6  4843680   4  98845931  80 types.CodeType
     7  30453   5  4385232   4 103231163  83 function
     8   4037   1  3813448   3 107044611  87 type
     9   1450   0  2279144   2 109323755  88 dict of module
<1436 more rows. Type e.g. '_.more' to view.>
end


In [4]:
df.head()

CHROM     POS ID REF ALT      QUAL FILTER VEP_Allele   VEP_Consequence  \
0  chr1   69428  .   T   G  10861.06      .          G  missense_variant   
1  chr1   69511  .   A   G   1479.06      .          G  missense_variant   
2  chr1  953279  .   T   C   3604.06      .          C  missense_variant   
3  chr1  976215  .   A   G    335.64      .          G  missense_variant   
4  chr1  978953  .   C   G    346.64      .          G  missense_variant   

  VEP_IMPACT  ... INFO_MQ INFO_QD INFO_SOR INFO_BaseQRankSum INFO_MQRankSum  \
0   MODERATE  ...   29.56   26.75    1.150            -2.239          1.312   
1   MODERATE  ...   27.90   27.91    5.709             2.569         -5.146   
2   MODERATE  ...   56.39   30.03    1.169              None           None   
3   MODERATE  ...   60.00    3.90    1.312            -1.583          0.000   
4   MODERATE  ...   60.00   12.84    1.179             2.656          0.000   

  INFO_ReadPosRankSum    QC_AF QC_GQ QC_GT QC_Weight  
0              -0.598  0.79628   1.0   2.0   1.59256  
1               0.818  0.00000   1.0   2.0   0.00000  
2                None  0.00000   1.0   2.0   0.00000  
3              -0.024  0.00000   1.0   1.0   0.00000  
4               0.753  0.00000   1.0   1.0   0.00000  

[5 rows x 75 columns]